This notebook assumes that you have a local instance of the TRAPI Pattern Fragment service running, which is loaded with the nodes/edges from ROBOKOP KG.

In [5]:
import requests,json
from datetime import datetime as dt

In [6]:
query_graph = {"nodes": {"subnode": {}, "objnode": {}},
                   "edges": {"the_edge": {"subject": "subnode", "object": "objnode"}}}

query_graph["edges"]["the_edge"]["predicates"] = ["biolink:affects"]
        
query_graph["edges"]["the_edge"]["qualifier_constraints"]= [
            {
                "qualifier_set": [
                    {
                        "qualifier_type_id": "biolink:object_aspect_qualifier",
                        "qualifier_value": "activity"
                    },
                    {
                        "qualifier_type_id": "biolink:object_direction_qualifier",
                        "qualifier_value": "decreased"
                    }
                ]
            }
]

query_graph ["nodes"]["subnode"]["ids"] = ["PUBCHEM.COMPOUND:70701426"]
query_graph ["nodes"]["objnode"]["categories"] = ["biolink:Gene"]

message = {"message":{"query_graph":query_graph}}

print(json.dumps(message,indent=2))

{
  "message": {
    "query_graph": {
      "nodes": {
        "subnode": {
          "ids": [
            "PUBCHEM.COMPOUND:70701426"
          ]
        },
        "objnode": {
          "categories": [
            "biolink:Gene"
          ]
        }
      },
      "edges": {
        "the_edge": {
          "subject": "subnode",
          "object": "objnode",
          "predicates": [
            "biolink:affects"
          ],
          "qualifier_constraints": [
            {
              "qualifier_set": [
                {
                  "qualifier_type_id": "biolink:object_aspect_qualifier",
                  "qualifier_value": "activity"
                },
                {
                  "qualifier_type_id": "biolink:object_direction_qualifier",
                  "qualifier_value": "decreased"
                }
              ]
            }
          ]
        }
      }
    }
  }
}


In [7]:
robokop_automat = "http://automat.renci.org/robokopkg/1.4/query"
robokop_tpf = "http://localhost:8000/query"

In [8]:
def time_test(input_message, query_url):
    now = dt.now()
    response = requests.post(query_url, json=message)
    later = dt.now()
    if response.status_code == 200:
        result = response.json()
        print(f"Returned {len(result['message']['results'])} in {later-now}")
        return result
    else:
        print(f"Status Code: {response.status_code}")
    return 

In [14]:
r = time_test(message,robokop_tpf)
for result in r["message"]["results"]:
    print(json.dumps(result["node_bindings"]["objnode"]))

Returned 5 in 0:00:00.064435
[{"id": "NCBIGene:246"}]
[{"id": "NCBIGene:239"}]
[{"id": "NCBIGene:239"}]
[{"id": "NCBIGene:11684"}]
[{"id": "NCBIGene:239"}]


In [15]:
r = time_test(message, robokop_automat)

Returned 3 in 0:00:00.194257
[{"id": "NCBIGene:246", "query_id": null, "attributes": null}]
[{"id": "NCBIGene:11684", "query_id": null, "attributes": null}]
[{"id": "NCBIGene:239", "query_id": null, "attributes": null}]


In [16]:
# Related to asthma

In [17]:
query_graph = {"nodes": {"subnode": {}, "objnode": {}},
                   "edges": {"the_edge": {"subject": "subnode", "object": "objnode"}}}

query_graph["edges"]["the_edge"]["predicates"] = ["biolink:related_to"]
        
query_graph ["nodes"]["subnode"]["ids"] = ["MONDO:0004979"]
query_graph ["nodes"]["objnode"]["categories"] = ["biolink:NamedThing"]

message = {"message":{"query_graph":query_graph}}

print(json.dumps(message,indent=2))

{
  "message": {
    "query_graph": {
      "nodes": {
        "subnode": {
          "ids": [
            "MONDO:0004979"
          ]
        },
        "objnode": {
          "categories": [
            "biolink:NamedThing"
          ]
        }
      },
      "edges": {
        "the_edge": {
          "subject": "subnode",
          "object": "objnode",
          "predicates": [
            "biolink:related_to"
          ]
        }
      }
    }
  }
}


In [18]:
r = time_test(message, robokop_automat)

Returned 5062 in 0:02:09.111197


In [69]:
rtpf = time_test(message, robokop_tpf)

Returned 6754 in 0:00:09.858578


In [70]:
onodes_automat = set([result["node_bindings"]["objnode"][0]["id"] for result in r["message"]["results"]])
onodes_tpf = set([result["node_bindings"]["objnode"][0]["id"] for result in rtpf["message"]["results"]])

In [72]:
len(onodes_automat)

4401

In [73]:
len(onodes_tpf)

4358

In [74]:
tpf_only = onodes_tpf.difference(onodes_automat)
len(tpf_only)

0

In [75]:
automat_only = onodes_automat.difference(onodes_tpf)
len(automat_only)

43

In [79]:
edge_ids = []
for result in r["message"]["results"]:
    if result["node_bindings"]["objnode"][0]["id"] in automat_only:
        edge_ids.append(result["analyses"][0]["edge_bindings"]["the_edge"][0]["id"])
print(len(edge_ids))

186


In [82]:
for eid in edge_ids:
    if not r["message"]["knowledge_graph"]["edges"][eid]["predicate"] == "biolink:subclass_of":
        print("found a missing result", eid)